<a href="https://colab.research.google.com/github/okana2ki/intro-to-DS/blob/main/DS13_2v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `SSDSE-C-2024.csv`の読み込み

In [ ]:
import pandas as pd

url = "https://www.nstac.go.jp/sys/files/SSDSE-C-2024.csv"
df = pd.read_csv(url, header=1, encoding="shift_jis")
print(df)

## AIへのデータ解説

**この後、AIにプログラム作成を依頼したときに、このデータを適切に解釈してもらえるように、↓下のセルで、コメント行の形で、このデータの形式と内容を（AIに対して）説明しておく。←これは重要です。これがあるのとないのとでは、適切なプログラムが生成される可能性が大きく変わるはず。**試しに、下記のセルよりも上のセルでAIに依頼して比べてみて下さい（AIは依頼したセルよりも下は読み込まないので）。（もちろん、プログラム作成を依頼するときに、このような情報を付けて依頼することもできる。）

↓下のセル内の
説明は、[サイト](https://www.nstac.go.jp/use/literacy/ssdse/) に掲載されている[データ解説PDFファイル](https://www.nstac.go.jp/sys/files/kaisetsu-C-2024.pdf) に基づいて書きました。（AIだけでなく）皆さんも、この解説PDFに目を通してからデータ分析するようにして下さい。

`header=1`と指定したため、変数`df`は、元のCSVファイルから、1行目を削除した内容になっていることに注意して下さい。

In [ ]:
# dfデータの解説
# データのレイアウト
# 1行目は項目名、それ以下の48行がデータ。データ行は、「全国」と「47都道府県庁所在市」に対応。
# 列は、最初の3列が地域情報を示す。1行目から順に「地域コード」「都道府県」「市」を示す。
# 4列目以降がデータで、4列目から順に「世帯人員」「食料（合計）」と続くが、5列目は、食品（合計）に対する1世帯当たり年間支出金額で、
# 6列目以降は、食品の品目ごとの1世帯当たり年間支出金額のデータである。
# なお、次の12個の列は、それぞれ、それ以降の列の小計になっている：「穀類」「魚介類」「肉類」「乳卵類」「野菜・海藻」「果物」「油脂・調味料」「菓子類」「調理食品」「飲料」「酒類」「外食」

## グラフ中の日本語表示を可能にする

AIに依頼すると、グラフ中に日本語表示するプログラムを作成することがあるため、グラフ中に日本語が表示できるように、あらかじめ、下記の2つのコードセルを実行しておきましょう。

In [ ]:
!pip install japanize_matplotlib

In [3]:
import japanize_matplotlib

# データのランキング

ランキングとは、順位付けです。たとえば、宮崎は餃子への支出額が一位とか、そういうやつです。



In [ ]:
# 数値データ列だけを対象に(numeric_only=True)、列ごとに降順でランキング
df_rank = df.rank(numeric_only=True, ascending=False)
print(df_rank)

In [ ]:
df_city = df[['市']]  # 上記のランキングには市の列が含まれないので、別途、市の列を抽出
print(df_city)

In [ ]:
# 市のDataFrameとランキングのDataFrameを連結→市の列を含むランキングが完成
df_city_rank = pd.concat([df_city, df_rank], axis=1)
print(df_city_rank)

In [ ]:
# prompt: df_city_rankから、宮崎市の行を抜き出して下さい。

# 宮崎市の行を抽出
df_miyazaki_rank = df_city_rank[df_city_rank['市'] == '宮崎市']

# 結果を表示
df_miyazaki_rank

In [ ]:
# prompt: 宮崎が上位１５位以内の品目名とランキングを抽出して下さい。

# 列名を取得
columns = df.columns[3:]  # dfの最初の3列は地域情報なので除外

# 上位15位以内の品目名とランキングを抽出
miyazaki_top15 = {}
for col in columns:
    rank = df_miyazaki_rank[col].iloc[0]
    if rank <= 15:
        miyazaki_top15[col] = rank

# 結果を表示
for item, rank in miyazaki_top15.items():
    print(f"品目名: {item}, ランキング: {rank}")

## 演習問題

同様に、宮崎が46位以降（少ない方から3位以内）の品目名とランキングを表示するプログラムを作って下さい。AIに頼んでもよいですが、自分でもできると思います。

`if rank <= 15:`

の代わりに、

`if rank >= 46:`

とすると良さそうですね。

In [ ]:
df_miyazaki_rank_t = df_miyazaki_rank.T
# .Tによる転置（行と列の入れ替え）←後の処理の都合で転置しておいた方が便利そうなので
print(df_miyazaki_rank_t)

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

df_miyazaki_rank_t.drop(df_miyazaki_rank_t.index[[0]]).plot.bar(figsize=(40,5))
# 数値データでない0行目を削除(.drop)して棒グラフを表示(.plot.bar)

## 演習問題
以上の結果から、宮崎は食料品消費額ランキング上位の品目は少なく、消費額ランキング下位の品目が多いことが分かりますね。この傾向は、食料（合計）のランキングが43位（下から6番目）であることにも表れています。

宮崎は物価が安いから消費額が少ないのか、年収が少ない人が多いから少ないのか、それぞれがどれくらい効いて、このような結果になっているのでしょうか？
[エンゲル係数](https://www.rakuten-card.co.jp/minna-money/topic/article_2103_00007/)の話は、これを考察する上で参考になりそうですね。
いずれにせよ、このことをちゃんと分析・考察するには、物価のデータや、年収のデータを合わせて調べる必要があります。
興味がある人は調べてみて下さい。

## 演習問題

**品目ごとの消費額を食料品全体の消費額（食料（合計））で割り算して、割合でランキング**すると、宮崎の特徴を表す別の傾向が見えそうです。ぜひ分析してみて下さい。地域ごとの物価の違いの影響をある程度抑えて、それぞれの市がどのような消費傾向を持っているかが見えるのではないかと思います。ただし、「宮崎ではピーマンが安い」のような個々の品目の価格の違いの影響は残ったままです。宮崎では全体として食品の物価が安いという傾向（があるとしたら、それ）は打ち消せるのですが。都市ごとの年収の違いの影響も抑えられるかな・・・

以下は、私がAIに依頼して作成した、割合でランキングするプログラムです。

In [ ]:
# prompt: 品目ごとの消費額を食料品全体の消費額（食料（合計））で割り算して、割合でランキングするプログラムを書いて下さい。

# 食料品全体の消費額（食料（合計））の列名を取得
total_food_expenditure_column = '食料（合計）'

# 各品目の消費額を食料品全体の消費額で割って割合を計算し、新しいDataFrameを作成
df_ratio = df.copy()  # 元のDataFrameをコピー
for col in df.columns[5:]:  # 6列目以降（品目ごとの消費額）をループ
    df_ratio[col] = df_ratio[col] / df_ratio[total_food_expenditure_column]

# 割合でランキング
df_ratio_rank = df_ratio.rank(numeric_only=True, ascending=False)

# 市の列を追加
df_city_ratio_rank = pd.concat([df['市'], df_ratio_rank], axis=1)

# 結果を表示
print(df_city_ratio_rank)

# 宮崎市の割合ランキングを表示
df_miyazaki_ratio_rank = df_city_ratio_rank[df_city_ratio_rank['市'] == '宮崎市']
print(df_miyazaki_ratio_rank)

# 上位15位と下位3位の品目とランキングを抽出
miyazaki_top15 = {}
miyazaki_bottom3 = {}
columns = df.columns[5:] # 品目ごとの列名を取得
for col in columns:
    rank = df_miyazaki_ratio_rank[col].iloc[0]
    if rank <= 15:
        miyazaki_top15[col] = rank
    elif rank >= 46 : #下位3位以内
        miyazaki_bottom3[col] = rank

print("宮崎市における上位15位の品目と割合ランキング")
for item, rank in miyazaki_top15.items():
    print(f"品目名: {item}, ランキング: {rank}")

print("\n宮崎市における下位3位の品目と割合ランキング")
for item, rank in miyazaki_bottom3.items():
    print(f"品目名: {item}, ランキング: {rank}")

# 可視化
df_miyazaki_ratio_rank_t = df_miyazaki_ratio_rank.T
df_miyazaki_ratio_rank_t.drop(df_miyazaki_ratio_rank_t.index[[0]]).plot.bar(figsize=(40,5))
plt.title("宮崎市における品目ごとの消費割合ランキング") #グラフにタイトルを追加
plt.ylabel("割合ランキング")  #y軸ラベルを追加
plt.show() #グラフを表示



---



ここで、クイズを出してくれるようインストラクションを書いたコードセルを挿入しておきましょう。クイズがじゃまだったら、↓下のコードセルを消すか、下方に移動して下さい。下方に移動するとそこより下のAIへの依頼のときだけ、このインストラクションが参照されます。

セルの消去は、セルをクリックして右上に現れるアイコンの中から「ゴミ箱」をクリック。下方への移動は、現れたアイコンの中から「↓」をクリック。

In [ ]:
# system instruction:
# ユーザはプログラミング初心者です。
# あなたの役割は、ユーザのコーディング依頼に応じる際、ユーザがプログラミングを習得できるようにすることです。
# ユーザのコーディング依頼に対してコードを生成する場合、依頼されたコードの実行後に、
# 「Pythonコードの書き方に関してユーザは何が分かってないかを自覚させるquiz」を日本語で出題するコードを生成して下さい。
# quizの解答に応じてユーザに日本語でフィードバックするコードも追加して下さい



---



# ヒストグラム
チューハイ・カクテルへの支出金額のヒストグラム

In [ ]:
import pandas as pd
import matplotlib.pylab as plt

file_name = "/content/SSDSE-C-2023.csv"
df = pd.read_csv(file_name, header=[0,1], encoding="shift_jis")
# print(df)

print(df.filter(like='チューハイ', axis=1))  # ヘッダーに'チューハイ'を含む列を検索

plt.hist(df[('LB110007','チューハイ・カクテル')])

# 散布図
りんご×オレンジの散布図

In [ ]:
!pip install japanize-matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

file_name = "/content/SSDSE-C-2023.csv"
df = pd.read_csv(file_name, header=[0,1], encoding="shift_jis")
# print(df)

# print(df.filter(like='りんご', axis=1))  # ヘッダーに'りんご'を含む列を検索
# print(df.filter(like='オレンジ', axis=1))  # ヘッダーに'オレンジ'を含む列を検索

x = df[('LB061001','りんご')]
y = df[('LB061004','オレンジ')]
city = df[('City','市')]
# print(city)

plt.figure(figsize=(15, 12))
plt.scatter(x,y) # 散布図
for i in range(len(x)): # 各点に市のラベルを付ける
    # 文字が重なってしまう都道府県名の表示位置を調整
    if city[i] in ['神戸市', '千葉市', '徳島市', '長崎市', '鹿児島市', '高松市', '宇都宮市', '名古屋市']:
        plt.text(x[i], y[i], city[i], ha='left', va='bottom')
    else:
        plt.text(x[i], y[i], city[i], ha='right', va='bottom')

plt.xlabel('りんごへの支出額')
plt.ylabel('オレンジへの支出額')
plt.show()

# df_city_ringo_orange = df[[('City','市'),('LB061001','りんご'),('LB061004','オレンジ')]]  # 市の列とりんごの列とオレンジの列を抽出
# print(df_city_ringo_orange)

# 参考
以下は、「データのランキング」のセクション後半で、支出額を割合で上書きするプログラムの作成時に、その動作を小さいDataFrameで確認したときのもの。参考までに掲載します。

なお、本日の小テストはこの（下記の）プログラムに関する問題です。**わらかないときは、生成AIに尋ねると教えてくれるはずですが、答を尋ねるのでなく、プログラムの分からない点を質問したりする方をお勧めします。その方が力がつくので。答をすぐに教えることを避ける設定にしてあると、そのように対応してくれるかもしれません。**

In [ ]:
import pandas as pd

dfs = pd.DataFrame({'a': [1,2], 'b': [3,4], 'c': [5,6]})
print(dfs)

for column_name, item in dfs.iloc[:,1:3].items():
  dfs[column_name] = dfs[column_name] / dfs['a']
print(dfs)